## Model Data using Cassandra

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [4]:
import pandas as pd
import numpy as np
import cassandra
import csv


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [ ]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries

In [1]:
! pip install cassandra-driver

In [26]:
import cassandra

In [27]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create & Set Keyspace

In [28]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS nosqldb 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [29]:
try:
    session.set_keyspace('nosqldb')
except Exception as e:
    print(e)

In [24]:
event_data = pd.read_csv('event_data.csv')
event_data.head(5)

,artist_name,fname,gender,item_in_session_number,lname,length,level,location,session_number,song_title,user_id
0,Pavement,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",345,Mercy:The Laundromat,10
1,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",438,Horn Concerto No. 4 in E flat K495: II. Romanc...,53
2,Gary Allan,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",438,Nothing On But The Radio,53
3,Charttraxx Karaoke,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",438,Fireflies,53
4,The Libertines,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",389,The Good Old Days,29


## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [33]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command   
query = "CREATE TABLE IF NOT EXISTS songs_played"
query = query + "(session_number int, item_in_session_number int, song_title text, artist_name text, length float, PRIMARY KEY (session_number, item_in_session_number))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [34]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO songs_played(session_number, item_in_session_number, song_title, artist_name, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
      
        session.execute(query, (int(row[8]), int(row[3]), row[9], row[0], float(row[5])))
    

### Validate our Data Model using a SELECT

In [48]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT artist_name, song_title, length FROM songs_played WHERE session_number= %s AND item_in_session_number = %s"
try:
    rows = session.execute(query, (338, 4))
except Exception as e:
    print(e)

for row in rows:
    print ("Artist Name:", row.artist_name, ", Song Title:", row.song_title, ", Length of song:", row.length)

Artist Name: Faithless , Song Title: Music Matters (Mark Knight Dub) , Length of song: 495.30731201171875


### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [49]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command    

query = "CREATE TABLE IF NOT EXISTS user_session_number"
query = query + "(user_id int, session_number int, item_in_session_number int, artist_name text, song_title text, fname text, lname text, PRIMARY KEY ((user_id, session_number), item_in_session_number))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [53]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO user_session_number (session_number, item_in_session_number, song_title, artist_name, fname, lname, user_id)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (int(row[8]), int(row[3]), row[9], row[0], row[1], row[4], int(row[10])))

### Validate our Data Model using a SELECT

In [54]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT artist_name, song_title, fname, lname FROM user_session_number WHERE user_id=%s AND session_number=%s"

try:
    rows = session.execute(query, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print ("Artist Name:", row.artist_name, ", Title of Song:", row.song_title, ", First name:", row.fname, ", Last Name:", row.lname)

Artist Name: Down To The Bone , Title of Song: Keep On Keepin' On , First name: Sylvie , Last Name: Cruz
Artist Name: Three Drives , Title of Song: Greece 2000 , First name: Sylvie , Last Name: Cruz
Artist Name: Sebastien Tellier , Title of Song: Kilometer , First name: Sylvie , Last Name: Cruz
Artist Name: Lonnie Gordon , Title of Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , First name: Sylvie , Last Name: Cruz


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [63]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command 

query = "CREATE TABLE IF NOT EXISTS user_song_listeners"
query = query + "(song_title text, user_id int, fname text, lname text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [65]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO user_song_listeners (song_title, user_id, fname, lname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[9], int(row[10]), row[1], row[4]))

### Validate our Data Model using a SELECT

In [66]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT fname, lname FROM user_song_listeners WHERE song_title=%s"
try:
    rows = session.execute(query, ("All Hands Against His Own",))
except Exception as e:
    print(e)
    
for row in rows:
    print ("First name:", row.fname, ", Last name:", row.lname)

First name: Jacqueline , Last name: Lynch
First name: Tegan , Last name: Levine
First name: Sara , Last name: Johnson


### Drop the tables before closing out the sessions

In [67]:
try:
    session.execute("DROP TABLE IF EXISTS table1")
    session.execute("DROP TABLE IF EXISTS table2")
    session.execute("DROP TABLE IF EXISTS table3")
except Exception as e:
    print (e)

### Close the session and cluster connection¶

In [68]:
session.shutdown()
cluster.shutdown()